In [25]:
import ccxt
from datetime import datetime
import pandas as pd
from sympy import Limit
import numpy as np
import time



## Fetch Ohlcv

In [26]:


def fetchData(exchange, symbol, timeframe, since="26-04-2022", limit=100):
    exchange_list = [
        'aax', 
        'ascendex', 
        'bequant', 
        'bibox',
        'bigone',
        'binance',
        'binancecoinm',
        'binanceus',
        'binanceusdm', 
        'bit2c', 
        'bitbank',
        'bitbay',
        'bitbns',
        'bitcoincom', 
        'bitfinex',
        'bitfinex2',
        'bitflyer', 
        'bitforex', 
        'bitget', 
        'bithumb', 
        'bitmart',
        'bitmex',
        'bitopro',
        'bitpanda', 
        'bitrue', 
        'bitso',
        'bitstamp', 
        'bitstamp1', 'bittrex', 'bitvavo', 'bkex', 'bl3p', 'blockchaincom', 'btcalpha', 'btcbox', 'btcmarkets', 'btctradeua', 'btcturk', 'buda', 'bw', 'bybit', 'bytetrade', 'cdax', 'cex', 'coinbase', 'coinbaseprime', 'coinbasepro', 'coincheck', 'coinex', 'coinfalcon', 'coinmate', 'coinone', 'coinspot', 'crex24', 'cryptocom', 'currencycom', 'delta', 'deribit', 'digifinex', 'eqonex', 'exmo', 'flowbtc', 'fmfwio', 'ftx', 'ftxus', 'gateio', 'gemini', 'hitbtc', 'hitbtc3', 'hollaex', 'huobi', 'huobijp', 'huobipro', 'idex', 'independentreserve', 'indodax', 'itbit', 'kraken', 'kucoin', 'kucoinfutures', 'kuna', 'latoken', 'lbank', 'liquid', 'luno', 'lykke', 'mercado', 'mexc', 'ndax', 'novadax', 'oceanex', 'okcoin', 'okex', 'okex5', 'okx', 'paymium', 'phemex', 'poloniex', 'probit', 'qtrade', 'ripio', 'stex', 'therock', 'tidebit', 'tidex', 'timex', 'upbit', 'vcc', 'wavesexchange', 'wazirx', 'whitebit', 'woo', 'xena', 'yobit', 'zaif', 'zb', 'zipmex', 'zonda']
    try:
        exchange = getattr(ccxt, exchange)()
    except AttributeError:
        print("-" * 36, " ERROR ", "-" * 35)
        print(
            'Exchange "{}" not found. Please check the exchange is supported.'.format(
                exchange
            )
        )
        print("Supported exchanges are:")
        print(exchange_list)
        print("-" * 80)
        quit()
    data = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
    header = ["Timestamp", "Open", "High", "Low", "Close", "Volume"]
    df = pd.DataFrame(data, columns=header)

    df.Timestamp = (
        df.Timestamp / 1000
    ) 
    df["Timestamp"] = pd.to_datetime(df.Timestamp, unit="s")

    df = df[["Timestamp", "Open", "High", "Low", "Close", "Volume"]]

    df["Open"] = pd.to_numeric(df["Open"])
    df["High"] = pd.to_numeric(df["High"])
    df["Low"] = pd.to_numeric(df["Low"])
    df["Close"] = pd.to_numeric(df["Close"])
    df["Volume"] = pd.to_numeric(df["Volume"])

    return df

In [27]:
print(fetchData("kraken","BTC/USDT","1m",30))

             Timestamp     Open     High      Low    Close    Volume
0  2022-04-28 08:27:00  39495.3  39495.3  39495.3  39495.3  0.000000
1  2022-04-28 08:28:00  39563.7  39567.2  39563.7  39567.2  1.000000
2  2022-04-28 08:29:00  39567.2  39567.2  39567.2  39567.2  0.000000
3  2022-04-28 08:30:00  39581.6  39587.0  39571.0  39571.0  0.112755
4  2022-04-28 08:31:00  39571.0  39571.0  39571.0  39571.0  0.000000
..                 ...      ...      ...      ...      ...       ...
95 2022-04-28 10:02:00  39827.2  39827.2  39810.1  39817.7  0.200659
96 2022-04-28 10:03:00  39817.7  39817.7  39817.7  39817.7  0.000000
97 2022-04-28 10:04:00  39817.7  39817.7  39817.7  39817.7  0.000000
98 2022-04-28 10:05:00  39801.3  39811.2  39801.3  39811.2  0.004215
99 2022-04-28 10:06:00  39832.2  39840.1  39832.2  39840.1  0.025690

[100 rows x 6 columns]


## Funciones-Fecthc Order Book para los distintos apis

In [28]:
def tim():
    t=time.localtime(time.time())
    return str(t[0])+"-"+str(t[1])+"-"+str(t[2])
tim()

'2022-4-28'

In [29]:
def fetch_order_book_bitso(symbol,limit):
    exchange = ccxt.bitso()
    bi_btc_ob = exchange.fetch_order_book(symbol,limit=limit)
    bi_btc_ob_time=bi_btc_ob["datetime"]
    bi_btc_ob_ask=pd.DataFrame(bi_btc_ob["asks"],columns=["Ask","AVolume"])
    bi_btc_ob_bid=pd.DataFrame(bi_btc_ob["bids"],columns=["Bid","BVolume"])
    dfk = pd.concat([bi_btc_ob_ask,  bi_btc_ob_bid],axis=1)
    dfk["Timestamp"]=bi_btc_ob_time
    dfk['Vwap Ask'] =(np.cumsum(dfk.Ask*dfk.AVolume) / np.cumsum(dfk.AVolume))
    dfk['Vwap Bid'] =(np.cumsum(dfk.Bid*dfk.BVolume) / np.cumsum(dfk.BVolume))
    dfk["Total Volume"]=(dfk.AVolume + dfk.BVolume)
    dfk["Midprice"]=((dfk.Ask + dfk.Bid)/2)
    dfk['Vwap Ask'] =(np.cumsum(dfk.Ask*dfk.AVolume) / np.cumsum(dfk.AVolume))
    dfk['Vwap Bid'] =(np.cumsum(dfk.Bid*dfk.BVolume) / np.cumsum(dfk.BVolume))
    dfk["Spread"]=(dfk.Bid - dfk.Ask)
    
    
                       
    return dfk


In [30]:
def fetch_order_book_ascendex(symbol,limit):
    exchange = ccxt.ascendex()
    bi_btc_ob = exchange.fetch_order_book(symbol,limit=limit)
    bi_btc_ob_time=bi_btc_ob["datetime"]
    bi_btc_ob_ask=pd.DataFrame(bi_btc_ob["asks"],columns=["Ask","AVolume"])
    bi_btc_ob_bid=pd.DataFrame(bi_btc_ob["bids"],columns=["Bid","BVolume"])
    dfk = pd.concat([bi_btc_ob_ask,  bi_btc_ob_bid],axis=1)
    dfk["Timestamp"]=bi_btc_ob_time
    dfk['Vwap Ask'] =(np.cumsum(dfk.Ask*dfk.AVolume) / np.cumsum(dfk.AVolume))
    dfk['Vwap Bid'] =(np.cumsum(dfk.Bid*dfk.BVolume) / np.cumsum(dfk.BVolume))
    dfk["Total Volume"]=(dfk.AVolume + dfk.BVolume)
    dfk["Midprice"]=((dfk.Ask + dfk.Bid)/2)
    dfk['Vwap Ask'] =(np.cumsum(dfk.Ask*dfk.AVolume) / np.cumsum(dfk.AVolume))
    dfk['Vwap Bid'] =(np.cumsum(dfk.Bid*dfk.BVolume) / np.cumsum(dfk.BVolume))
    dfk["Spread"]=(dfk.Bid - dfk.Ask)
    
    
                       
    return dfk


In [31]:
def fetch_order_book_bitbay(symbol,limit):
    exchange = ccxt.bitbay()
    bi_btc_ob = exchange.fetch_order_book(symbol,limit=limit)
    bi_btc_ob_time=bi_btc_ob["datetime"]
    bi_btc_ob_ask=pd.DataFrame(bi_btc_ob["asks"],columns=["Ask","AVolume"])
    bi_btc_ob_bid=pd.DataFrame(bi_btc_ob["bids"],columns=["Bid","BVolume"])
    dfk = pd.concat([bi_btc_ob_ask,  bi_btc_ob_bid],axis=1)
    dfk["Timestamp"]=bi_btc_ob_time
    dfk['Vwap Ask'] =(np.cumsum(dfk.Ask*dfk.AVolume) / np.cumsum(dfk.AVolume))
    dfk['Vwap Bid'] =(np.cumsum(dfk.Bid*dfk.BVolume) / np.cumsum(dfk.BVolume))
    dfk["Total Volume"]=(dfk.AVolume + dfk.BVolume)
    dfk["Midprice"]=((dfk.Ask + dfk.Bid)/2)
    dfk['Vwap Ask'] =(np.cumsum(dfk.Ask*dfk.AVolume) / np.cumsum(dfk.AVolume))
    dfk['Vwap Bid'] =(np.cumsum(dfk.Bid*dfk.BVolume) / np.cumsum(dfk.BVolume))
    dfk["Spread"]=(dfk.Bid - dfk.Ask)
    
    
                       
    return dfk

In [32]:
def fetch_order_book_bitbns(symbol,limit):
    exchange = ccxt.bitbns()
    bi_btc_ob = exchange.fetch_order_book(symbol,limit=limit)
    bi_btc_ob_time=bi_btc_ob["datetime"]
    bi_btc_ob_ask=pd.DataFrame(bi_btc_ob["asks"],columns=["Ask","AVolume"])
    bi_btc_ob_bid=pd.DataFrame(bi_btc_ob["bids"],columns=["Bid","BVolume"])
    dfk = pd.concat([bi_btc_ob_ask,  bi_btc_ob_bid],axis=1)
    dfk["Timestamp"]=bi_btc_ob_time
    dfk['Vwap Ask'] =(np.cumsum(dfk.Ask*dfk.AVolume) / np.cumsum(dfk.AVolume))
    dfk['Vwap Bid'] =(np.cumsum(dfk.Bid*dfk.BVolume) / np.cumsum(dfk.BVolume))
    dfk["Total Volume"]=(dfk.AVolume + dfk.BVolume)
    dfk["Midprice"]=((dfk.Ask + dfk.Bid)/2)
    dfk['Vwap Ask'] =(np.cumsum(dfk.Ask*dfk.AVolume) / np.cumsum(dfk.AVolume))
    dfk['Vwap Bid'] =(np.cumsum(dfk.Bid*dfk.BVolume) / np.cumsum(dfk.BVolume))
    dfk["Spread"]=(dfk.Bid - dfk.Ask)
    
    
                       
    return dfk